# NTDS'18 milestone 1: network collection and properties
[Effrosyni Simou](https://lts4.epfl.ch/simou), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: `<your team number>`
* Students: `<the name of all students in the team>`
* Dataset: `<the dataset you used to complete the milestone>`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to three sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective 

The purpose of this milestone is to start getting acquainted to the network that you will use for this class. In the first part of the milestone you will import your data using [Pandas](http://pandas.pydata.org) and you will create the adjacency matrix using [Numpy](http://www.numpy.org). This part is project specific. In the second part you will have to compute some basic properties of your network. **For the computation of the properties you are only allowed to use the packages that have been imported in the cell below.** You are not allowed to use any graph-specific toolboxes for this milestone (such as networkx and PyGSP). Furthermore, the aim is not to blindly compute the network properties, but to also start to think about what kind of network you will be working with this semester. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Import json to retrieve the data
import json
#Import requests to send CURL requests to the API. This needs to be installed beforehand
import requests
#Import distances
from scipy.spatial.distance import pdist, squareform

## Part 1 - Import your data and manipulate them. 

###  A. Load your data in a Panda dataframe.

First, you should define and understand what are your nodes, what features you have and what are your labels. Please provide below a Panda dataframe where each row corresponds to a node with its features and labels. For example, in the the case of the Free Music Archive (FMA) Project, each row of the dataframe would be of the following form:


| Track   |  Feature 1  | Feature 2 | . . . | Feature 518|  Label 1 |  Label 2 |. . .|Label 16|
|:-------:|:-----------:|:---------:|:-----:|:----------:|:--------:|:--------:|:---:|:------:|
|         |             |           |       |            |          |          |     |        |

It is possible that in some of the projects either the features or the labels are not available. This is OK, in that case just make sure that you create a dataframe where each of the rows corresponds to a node and its associated features or labels.

In [2]:
#Define a reference function to retrieve data from the API. It returns the "results" object of the retrieved json file.
#You need an API key stored in the APIKey.txt file
def getFromAPI(url):
    with open ("APIKey.txt", "r") as keyFile:
        apiKey=keyFile.readline()
        if apiKey[-1] == '\n':
            apiKey = apiKey[:-1]
            
    headers = {'X-API-Key': apiKey}
    
    r = requests.get(url, headers = headers)
    r = r.json() #Parse the result to a json file
    #print(r['status'])#Should display "Ok"
    
    results = r['results'] #Retrieve the interesting part
    results = results[0]
    
    return results

#This function filters out the senators not in office anymore
def filterActiveSenators(senatorsList):
    senatorActiveList = []
    for senator in senatorsList:
        if senator["in_office"]:
            senatorActiveList.append(senator)
    nbActiveSenators = len(senatorActiveList)
    print("There are " + str(nbActiveSenators) + " active senators among the " + str(len(senatorsList)) + " senators")
    return senatorActiveList

In [3]:
# Parameters

requestsPerSenators = 40; #The number of considered votes will be 20*requestsPerSenators
weights_threshold = 0.5; #Threshold applied to the weights to determine if a link is to be made or not

In [ ]:
# Retrieve the list of all senators, filter it and save it
r = getFromAPI("https://api.propublica.org/congress/v1/115/senate/members.json")
senatorsList = r["members"]
senatorActiveList = filterActiveSenators(senatorsList)
DFsenators = pd.io.json.json_normalize(senatorActiveList)

np.save('senatorActiveList',senatorActiveList)

# Build the DataFrames : votesBySenators and votes
jDFvotesBySenators = [] #Dataframe built in a list of dictionnaries
jDFvotes = [] 
uniqueVotes = [];

for senator in senatorActiveList: # For each senator, retrieve his/her votes (features ) and his/her labels
    #For each senator, keep track of his votes in a dictionary and complete the dictionnary with the relevant labels
    senatorDict = senator
    for offset in range(0,requestsPerSenators*20,20): #Retrieve the votes 20 by 20
        r = getFromAPI("https://api.propublica.org/congress/v1/members/"+senator['id']+"/votes.json?offset="+str(offset))
        votes = r['votes'] #List of votes in this range
        for vote in votes: #For each vote in the range, append it to the dictionnary of the senator
            voteID = "S"+vote['session']+"C"+vote['roll_call']
            senatorDict[voteID]=vote['position']
            
            if voteID not in uniqueVotes: #If we never encountered that vote, add it to the jDFvotes
                del vote["member_id"]
                del vote["position"]
                vote["id"] = voteID
                uniqueVotes.append(voteID)
                jDFvotes.append(vote)
    jDFvotesBySenators.append(senatorDict)

DFvotes = pd.io.json.json_normalize(jDFvotes)
DFvotesBySenators = pd.io.json.json_normalize(jDFvotesBySenators)
DFvotesBySenators
DFvotesBySenators.to_pickle('./DFvotesBySenators.pkl')
DFvotes.to_pickle('./DFvotes.pkl')


There are 100 active senators among the 105 senators


In [ ]:
features = DFvotesBySenators[uniqueVotes]

labelsList = DFvotesBySenators.columns.tolist()
for x in uniqueVotes:
    labelsList.remove(x)
    
labels = DFvotesBySenators[labelsList]

In [ ]:
features

In [ ]:
labels

### B. Create the adjacency matrix of your network.

Remember that there are edges connecting the attributed nodes that you organized in the dataframe above. The connectivity of the network is captured by the adjacency matrix $W$. If $N$ is the number of nodes, the adjacency matrix is an $N \times N$ matrix where the value of $W(i,j)$ is the weight of the edge connecting node $i$ to node $j$.  

There are two possible scenarios for your adjacency matrix construction, as you already learned in the tutorial by Benjamin:

1) The edges are given to you explicitly. In this case you should simply load the file containing the edge information and parse it in order to create your adjacency matrix. See how to do that in [this tutorial](../tutorials/02a_graph_from_edge_list.ipynb).

2) The edges are not given to you. In that case you will have to create a feature graph. In order to do that you will have to chose a distance that will quantify how similar two nodes are based on the values in their corresponding feature vectors. [This tutorial](../tutorials/02b_graph_from_features.ipynb) shows you how to build feature graphs when using Euclidean distances between feature vectors. Be curious and explore other distances as well! For instance, in the case of high-dimensional feature vectors, you might want to consider using the cosine distance. Once you compute the distances between your nodes you will have a fully connected network. Do not forget to sparsify by keeping the most important edges in your network.

Follow the appropriate steps for the construction of the adjacency matrix of your network and provide it in the Numpy array ``adjacency`` below: 

In [ ]:
#To avoid re-launch the script and all the API queries:
features = pd.read_csv('features.csv')
labels = pd.read_csv('labels.csv')

In [ ]:
#Compute the weigths of the network

#Convert features to numbers
features = features.replace('Yes',1)
features = features.replace('No',0)

#All others values should be NaN
cols= features.columns
features[cols] = features[cols].apply(pd.to_numeric, errors='coerce')


#Define a distance ignoring the NaN values
def euclidian_without_NaN(x, y):
    return  np.sqrt(np.nansum(((x-y)**2)))

def l1_normalized_without_NaN(x, y):
    return  np.nansum((np.absolute(x-y)))/np.count_nonzero(~np.isnan(x-y))

distances = pdist(features.values, l1_normalized_without_NaN)

#Distances to weights
kernel_width = distances.mean()
weights = np.exp(-distances**2 / kernel_width**2)



# Turn the list of weights into a matrix.
adjacency = squareform(weights)
#Apply a threshold to the weights to avoid a completely connected network
plt.hist(weights)
plt.title('Distribution of weights')
plt.show()
adjacency[adjacency < weights_threshold] = 0


print(adjacency)

n_nodes = np.sqrt(adjacency.size)
print(n_nodes)

## Part 2

Execute the cell below to plot the (weighted) adjacency matrix of your network.

In [ ]:
plt.spy(adjacency, markersize=1)
plt.title('adjacency matrix')

### Question 1

What is the maximum number of links $L_{max}$ in a network with $N$ nodes (where $N$ is the number of nodes in your network)? How many links $L$ are there in your collected network? Comment on the sparsity of your network.

In [ ]:
L = np.count_nonzero(adjacency)/2
print("There are "+str(L)+" links in the network")

**Your answer here.**

The maximum number of links in an undirected newtork with N nodes, without self-links is N*(N-1)/2. With our network, this means up to 4950 links. We have 2185 links in our network after the thresholding.


### Question 2

Is your graph directed or undirected? If it is directed, convert it to an undirected graph by symmetrizing the adjacency matrix.

**Your answer here.**

Our network is already undirected

### Question 3

In the cell below save the features dataframe and the **symmetrized** adjacency matrix. You can use the Pandas ``to_csv`` to save the ``features`` and Numpy's ``save`` to save the ``adjacency``. We will reuse those in the following milestones.

In [ ]:
features.to_csv('features.csv')
labels.to_csv('labels.csv')
np.save('adjacency',adjacency)

### Question 4

Are the edges of your graph weighted?

**Your answer here.**

Yes, the weights were constructed from the euclidian distances between the features of each senator.

### Question 5

What is the degree distibution of your network? 

In [ ]:
degree =  np.sum(adjacency, axis=0)

assert len(degree) == n_nodes

Execute the cell below to see the histogram of the degree distribution.

In [ ]:
weights = np.ones_like(degree) / float(n_nodes)
plt.hist(degree, weights=weights);

What is the average degree?

In [ ]:
np.mean(degree)

### Question 6

Comment on the degree distribution of your network.

**Your answer here.**

The majority of the nodes have a degree close to the average one (between 30 and 45). A few nodes have a very low degree, meaning some nodes are more isolated than the average node. 

### Question 7

Write a function that takes as input the adjacency matrix of a graph and determines whether the graph is connected or not.

In [ ]:
def connected_graph(adjacency):
    """Determines whether a graph is connected.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    bool
        True if the graph is connected, False otherwise.
    """
    
    # Your code here.
    numOfNodes = adjacency.shape[0]
    
    # Implementation of BFS algorithm
    visited = np.zeros(numOfNodes)
    visited[0] = 1
    queue = np.array([0])
    while (queue.size != 0):
        head = queue[0]
        queue = np.delete(queue, 0)
        for nodeId, isNeibor in enumerate(adjacency[head]):
            if (visited[nodeId] == 0 and isNeibor):
                queue = np.append(queue, nodeId)
                visited[nodeId] = 1
            
    
    
    print(visited)
    return visited.sum() == numOfNodes

Is your graph connected? Run the ``connected_graph`` function to determine your answer.

In [ ]:
# Your code here.
connected_graph(adjacency)

The graph is not connencted, senator n°54 has no edges

### Question 8

Write a function that extracts the connected components of a graph.

In [ ]:
def find_components(adjacency):
    """Find the connected components of a graph.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    list of numpy arrays
        A list of adjacency matrices, one per connected component.
    """
    
    
    return components

How many connected components is your network composed of? What is the size of the largest connected component? Run the ``find_components`` function to determine your answer. 

In [ ]:
# Your code here.

### Question 9

Write a function that takes as input the adjacency matrix and a node (`source`) and returns the length of the shortest path between that node and all nodes in the graph using Dijkstra's algorithm. **For the purposes of this assignment we are interested in the hop distance between nodes, not in the sum of weights. **

Hint: You might want to mask the adjacency matrix in the function ``compute_shortest_path_lengths`` in order to make sure you obtain a binary adjacency matrix. 

In [ ]:
def adjacencyToBinary(adjacency):
    """ Returns the sign of each element in the adjacency matrix"""
    return np.sign(adjacency)

def compute_shortest_path_lengths(adjacency, source):
    """Compute the shortest path length between a source node and all nodes.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    
    Returns
    -------
    list of ints
        The length of the shortest path from source to all nodes. Returned list should be of length n_nodes.
    """
    # Find the shortest path using DIJSKTRA's algorithm
    numOfNodes = adjacency.shape[0]
    adjacencyBinary = adjacencyToBinary(adjacency)
    dist = np.zeros(numOfNodes)
    dist += float('inf')
    prev = -np.ones(numOfNodes)
    dist[source] = 0
    distAux = dist.copy()
    vertex = np.arange(numOfNodes)
    
    while vertex.size > 0 and np.min(distAux) < float('inf'):
        node = np.argmin(distAux)
        vertex = np.delete(vertex, np.where(vertex == node))
        
        for ind, n in enumerate(adjacencyBinary[node]):
            if (n and (dist[node] + 1 < dist[ind])):
                dist[ind] = dist[node] + 1
                distAux[ind] = distAux[node] + 1
                prev[ind] = node
    
        # Mark node as visited
        distAux[node] = float('inf')
    dist[dist >= 10**300] = float('inf')
    return dist

In [ ]:
compute_shortest_path_lengths(adjacency, 0)

### Question 10

The diameter of the graph is the length of the longest shortest path between any pair of nodes. Use the above developed function to compute the diameter of the graph (or the diameter of the largest connected component of the graph if the graph is not connected). If your graph (or largest connected component) is very large, computing the diameter will take very long. In that case downsample your graph so that it has 1.000 nodes. There are many ways to reduce the size of a graph. For the purposes of this milestone you can chose to randomly select 1.000 nodes. 

In [ ]:
# Your code here.

### Question 11

Write a function that takes as input the adjacency matrix, a path length, and two nodes (`source` and `target`), and returns the number of paths of the given length between them.

In [ ]:
def compute_paths(adjacency, source, target, length):
    """Compute the number of paths of a given length between a source and target node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    target: int
        The target node. A number between 0 and n_nodes-1.
    length: int
        The path length to be considered.
    
    Returns
    -------
    int
        The number of paths.
    """
    
    # Your code here.
    
    return n_paths

Test your function on 5 pairs of nodes, with different lengths.

In [ ]:
print(compute_paths(adjacency, 0, 10, 1))
print(compute_paths(adjacency, 0, 10, 2))
print(compute_paths(adjacency, 0, 10, 3))
print(compute_paths(adjacency, 23, 67, 2))
print(compute_paths(adjacency, 15, 93, 4))

### Question 12

How many paths of length 3 are there in your graph? Hint: calling the `compute_paths` function on every pair of node is not an efficient way to do it.

In [ ]:
# Your code here.

### Question 13

Write a function that takes as input the adjacency matrix of your graph (or of the largest connected component of your graph) and a node and returns the clustering coefficient of that node. 

In [ ]:
def compute_clustering_coefficient(adjacency, node):
    """Compute the clustering coefficient of a node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    node: int
        The node whose clustering coefficient will be computed. A number between 0 and n_nodes-1.
    
    Returns
    -------
    float
        The clustering coefficient of the node. A number between 0 and 1.
    """
    
    # Your code here.
    
    return clustering_coefficient

### Question 14

What is the average clustering coefficient of your graph (or of the largest connected component of your graph if your graph is disconnected)? Use the function ``compute_clustering_coefficient`` to determine your answer.

In [ ]:
# Your code here.